## 테스트 데이터셋 로드

In [1]:
import os
import urllib.request
import tarfile
import pickle
import numpy as np
from common.ResNet20 import ResNet20

def download_cifar100(save_path='cifar-100-python'):
    if os.path.exists(save_path):
        print("CIFAR-100 이미 존재")
        return

    url = 'https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz'
    filename = 'cifar-100-python.tar.gz'
    print("CIFAR-100 다운로드 중...")
    urllib.request.urlretrieve(url, filename)

    with tarfile.open(filename, 'r:gz') as tar:
        tar.extractall()
    os.remove(filename)
    print("다운로드 완료")

def load_batch(filepath):
    with open(filepath, 'rb') as f:
        data_dict = pickle.load(f, encoding='bytes')
    data = data_dict[b'data']
    fine_labels = np.array(data_dict[b'fine_labels'])
    coarse_labels = np.array(data_dict[b'coarse_labels'])
    data = data.reshape(-1, 3, 32, 32).astype(np.float32) / 255.0
    return data, fine_labels, coarse_labels

In [2]:
def load_cifar100_full(data_dir='./cifar-100-python', valid_ratio=0.1):
    # 1. Load train and test batch
    x_train_all, y_train_fine_all, y_train_coarse_all = load_batch(os.path.join(data_dir, 'train'))
    x_test, y_test_fine, y_test_coarse = load_batch(os.path.join(data_dir, 'test'))

    # 2. Split train → train + validation
    num_total = x_train_all.shape[0]
    num_valid = int(num_total * valid_ratio)

    x_valid = x_train_all[:num_valid]
    t_valid_fine = y_train_fine_all[:num_valid]
    t_valid_coarse = y_train_coarse_all[:num_valid]

    x_train = x_train_all[num_valid:]
    t_train_fine = y_train_fine_all[num_valid:]
    t_train_coarse = y_train_coarse_all[num_valid:]

    # 3. Return all
    return (x_train, x_valid, x_test,
            t_train_coarse, t_valid_coarse, y_test_coarse,
            t_train_fine, t_valid_fine, y_test_fine)

In [3]:
# CIFAR-100 다운로드
download_cifar100()

# 전체 데이터셋 로드
(x_train, x_valid, x_test,
 t_train_coarse, t_valid_coarse, t_test_coarse,
 t_train_fine, t_valid_fine, t_test_fine) = load_cifar100_full()

# 확인 출력
print("CIFAR-100 데이터셋 로드 완료:")
print(" - x_train:", x_train.shape)
print(" - x_valid:", x_valid.shape)
print(" - x_test :", x_test.shape)
print(" - t_train_fine :", t_train_fine.shape)
print(" - t_valid_fine :", t_valid_fine.shape)
print(" - t_test_fine  :", t_test_fine.shape)
print(" - t_train_coarse:", t_train_coarse.shape)
print(" - t_valid_coarse:", t_valid_coarse.shape)
print(" - t_test_coarse :", t_test_coarse.shape)


CIFAR-100 이미 존재
CIFAR-100 데이터셋 로드 완료:
 - x_train: (45000, 3, 32, 32)
 - x_valid: (5000, 3, 32, 32)
 - x_test : (10000, 3, 32, 32)
 - t_train_fine : (45000,)
 - t_valid_fine : (5000,)
 - t_test_fine  : (10000,)
 - t_train_coarse: (45000,)
 - t_valid_coarse: (5000,)
 - t_test_coarse : (10000,)


In [4]:
def restore_model_parameters(model, model_state):
    
    model.conv1.W = model_state['conv1_W']
    model.conv1.b = model_state['conv1_b']

    model.fc.W = model_state['fc_W']
    model.fc.b = model_state['fc_b']

    idx = 0
    for block in model.layer1 + model.layer2 + model.layer3:
        for attr in ['conv1', 'conv2', 'shortcut']:
            if hasattr(block, attr):
                conv = getattr(block, attr)
                conv.W = model_state[f'{idx}_W']
                conv.b = model_state[f'{idx}_b']
                idx += 1

def restore_bn_params(model, state):
    bn_count = 0
    for block in model.layer1 + model.layer2 + model.layer3:
        for attr in ['bn1', 'bn2']:
            bn = getattr(block, attr)
            bn.gamma = state[f'{bn_count}_gamma']
            bn.beta = state[f'{bn_count}_beta']
            bn.running_mean = state[f'{bn_count}_running_mean']
            bn.running_var = state[f'{bn_count}_running_var']
            bn_count += 1
        if hasattr(block, 'bn_shortcut'):
            bn = block.bn_shortcut
            bn.gamma = state[f'{bn_count}_gamma']
            bn.beta = state[f'{bn_count}_beta']
            bn.running_mean = state[f'{bn_count}_running_mean']
            bn.running_var = state[f'{bn_count}_running_var']
            bn_count += 1

    bn = model.bn1
    bn.gamma = state[f'{bn_count}_gamma']
    bn.beta = state[f'{bn_count}_beta']
    bn.running_mean = state[f'{bn_count}_running_mean']
    bn.running_var = state[f'{bn_count}_running_var']

In [5]:
from common.ResNet20 import ResNet20

model = ResNet20()

with open("crop_epoch_10.pkl", "rb") as f:
    checkpoint = pickle.load(f)
    model_state = checkpoint["model"]
    
restore_bn_params(model, model_state)
restore_model_parameters(model, model_state)

In [6]:
print("fc_W std:", np.std(model.fc.W))
print("fc_b std:", np.std(model.fc.b))
print("conv1_W std:", np.std(model.conv1.W))

fc_W std: 0.6059813480004459
fc_b std: 0.2697215860402543
conv1_W std: 0.6362145435582259


In [7]:
from common.functions import softmax  

def evaluate_model(model, x, y_true):
    batch_size = 100
    preds = []
    for i in range(0, x.shape[0], batch_size):
        x_batch = x[i:i+batch_size]
        logits = model.predict(x_batch)        
        probs = softmax(logits)                
        y_pred = np.argmax(probs, axis=1)
        preds.append(y_pred)
    preds = np.concatenate(preds)
    acc = np.sum(preds == y_true) / len(y_true)
    return preds, acc

#pred : 예측된 클래스 번호
#전체 정확도

In [8]:
fine_preds, fine_acc = evaluate_model(model, x_train, t_train_fine)
print(f"[🎯 Fine Label] Train Accuracy: {fine_acc:.4f}")

[🎯 Fine Label] Train Accuracy: 0.5412


In [9]:
fine_valid_preds, fine_valid_acc = evaluate_model(model, x_valid, t_valid_fine)
print(f"[🎯 Fine Label] Valid Accuracy: {fine_valid_acc:.4f}")

[🎯 Fine Label] Valid Accuracy: 0.5408


In [22]:
fine_test_preds, fine_test_acc = evaluate_model(model, x_test, t_test_fine)
print(f"[🎯 Fine Label] Test Accuracy: {fine_test_acc:.4f}")

[🎯 Fine Label] Test Accuracy: 0.4826


## 매핑 전 필요한거

In [10]:
## 단순 딕셔너리
import pickle

def load_fine_to_coarse_from_train(data_dir='./cifar-100-python'):
    with open(f"{data_dir}/train", 'rb') as f:
        train_data = pickle.load(f, encoding='bytes')

    fine_labels = train_data[b'fine_labels']
    coarse_labels = train_data[b'coarse_labels']

    fine_to_coarse = {}
    for f, c in zip(fine_labels, coarse_labels):
        if f not in fine_to_coarse:
            fine_to_coarse[f] = c
        else:
            assert fine_to_coarse[f] == c, f"Conflict: {f} → {fine_to_coarse[f]} vs {c}"
    
    return fine_to_coarse


# 매핑 테이블
def get_cifar100_fine_to_coarse_dict():
    """
    CIFAR-100의 fine label (0~99)을 coarse label (0~19)로 매핑하는 딕셔너리를 반환
    """
    fine_to_coarse = {
         0:  0,  1:  0,  2:  0,  3:  0,  4:  0,
         5:  1,  6:  1,  7:  1,  8:  1,  9:  1,
        10:  2, 11:  2, 12:  2, 13:  2, 14:  2,
        15:  3, 16:  3, 17:  3, 18:  3, 19:  3,
        20:  4, 21:  4, 22:  4, 23:  4, 24:  4,
        25:  5, 26:  5, 27:  5, 28:  5, 29:  5,
        30:  6, 31:  6, 32:  6, 33:  6, 34:  6,
        35:  7, 36:  7, 37:  7, 38:  7, 39:  7,
        40:  8, 41:  8, 42:  8, 43:  8, 44:  8,
        45:  9, 46:  9, 47:  9, 48:  9, 49:  9,
        50: 10, 51: 10, 52: 10, 53: 10, 54: 10,
        55: 11, 56: 11, 57: 11, 58: 11, 59: 11,
        60: 12, 61: 12, 62: 12, 63: 12, 64: 12,
        65: 13, 66: 13, 67: 13, 68: 13, 69: 13,
        70: 14, 71: 14, 72: 14, 73: 14, 74: 14,
        75: 15, 76: 15, 77: 15, 78: 15, 79: 15,
        80: 16, 81: 16, 82: 16, 83: 16, 84: 16,
        85: 17, 86: 17, 87: 17, 88: 17, 89: 17,
        90: 18, 91: 18, 92: 18, 93: 18, 94: 18,
        95: 19, 96: 19, 97: 19, 98: 19, 99: 19
    }
    return fine_to_coarse


def map_softmax_to_coarse(softmax_output, fine_to_coarse):
    """
    softmax_output: shape (N, 100), softmax 확률 결과
    fine_to_coarse: dict, fine label → coarse label 매핑
    
    return: pred_coarse: shape (N,), coarse label 예측 결과
    """
    fine_pred = np.argmax(softmax_output, axis=1)  # shape (N,)
    coarse_pred = np.array([fine_to_coarse[f] for f in fine_pred])  # shape (N,)
    return coarse_pred

### valid에 대한 매핑

### test에 대한 매핑

In [11]:
#fine에 대한 softmax 값  "test"
logits = model.predict(x_test)        
softmax_output = softmax(logits)  

#매핑 기법별 예시 출력 할 때 필요 -> pred_fine/ true_fine/ pred_coarse/ true_coarse
# 1.pred_fine (argmax)
pred_fine = np.argmax(softmax_output, axis=1)
# 2. 정답 fine label
true_fine = t_test_fine  #!!!
# 3. fine → coarse 
#fine_to_coarse = 기법별로 ,,,
# 4. 정답 coarse label
true_coarse = t_test_fine
fine_to_coarse = get_cifar100_fine_to_coarse_dict()
true_coarse = np.array([fine_to_coarse[f] for f in true_fine])

## 1. 단순 매핑

In [12]:
## 단순 딕셔너리
import pickle

def load_fine_to_coarse_from_train(data_dir='./cifar-100-python'):
    with open(f"{data_dir}/train", 'rb') as f:
        train_data = pickle.load(f, encoding='bytes')

    fine_labels = train_data[b'fine_labels']
    coarse_labels = train_data[b'coarse_labels']

    fine_to_coarse = {}
    for f, c in zip(fine_labels, coarse_labels):
        if f not in fine_to_coarse:
            fine_to_coarse[f] = c
        else:
            assert fine_to_coarse[f] == c, f"Conflict: {f} → {fine_to_coarse[f]} vs {c}"
    
    return fine_to_coarse


# 매핑 테이블
def get_cifar100_fine_to_coarse_dict():
    """
    CIFAR-100의 fine label (0~99)을 coarse label (0~19)로 매핑하는 딕셔너리를 반환
    """
    fine_to_coarse = {
         0:  0,  1:  0,  2:  0,  3:  0,  4:  0,
         5:  1,  6:  1,  7:  1,  8:  1,  9:  1,
        10:  2, 11:  2, 12:  2, 13:  2, 14:  2,
        15:  3, 16:  3, 17:  3, 18:  3, 19:  3,
        20:  4, 21:  4, 22:  4, 23:  4, 24:  4,
        25:  5, 26:  5, 27:  5, 28:  5, 29:  5,
        30:  6, 31:  6, 32:  6, 33:  6, 34:  6,
        35:  7, 36:  7, 37:  7, 38:  7, 39:  7,
        40:  8, 41:  8, 42:  8, 43:  8, 44:  8,
        45:  9, 46:  9, 47:  9, 48:  9, 49:  9,
        50: 10, 51: 10, 52: 10, 53: 10, 54: 10,
        55: 11, 56: 11, 57: 11, 58: 11, 59: 11,
        60: 12, 61: 12, 62: 12, 63: 12, 64: 12,
        65: 13, 66: 13, 67: 13, 68: 13, 69: 13,
        70: 14, 71: 14, 72: 14, 73: 14, 74: 14,
        75: 15, 76: 15, 77: 15, 78: 15, 79: 15,
        80: 16, 81: 16, 82: 16, 83: 16, 84: 16,
        85: 17, 86: 17, 87: 17, 88: 17, 89: 17,
        90: 18, 91: 18, 92: 18, 93: 18, 94: 18,
        95: 19, 96: 19, 97: 19, 98: 19, 99: 19
    }
    return fine_to_coarse


def map_softmax_to_coarse(softmax_output, fine_to_coarse):
    """
    softmax_output: shape (N, 100), softmax 확률 결과
    fine_to_coarse: dict, fine label → coarse label 매핑
    
    return: pred_coarse: shape (N,), coarse label 예측 결과
    """
    fine_pred = np.argmax(softmax_output, axis=1)  # shape (N,)
    coarse_pred = np.array([fine_to_coarse[f] for f in fine_pred])  # shape (N,)
    return coarse_pred

In [13]:
## 사용
# 1. 모델 추론 후 softmax 출력 얻기
#softmax_output

# 2. 매핑 딕셔너리 불러오기
fine_to_coarse = get_cifar100_fine_to_coarse_dict()

# 3. coarse label 예측 #softmax 결과, 딕셔너리
y_coarse_dict = map_softmax_to_coarse(softmax_output, fine_to_coarse)

#예시출력
pred_coarse = y_coarse_dict


# --- 정확도 출력 ---
acc_dict = np.mean(y_coarse_dict == true_coarse)
print(f"argmax dictionary coarse accuracy: {acc_dict:.4f}\n")

# --- 예시 출력 ---
for a in range(30):
    print(f"[{a}] predict fine label:    {pred_fine[a]}")
    print(f"    true fine label:      {true_fine[a]}")
    print(f"    predict coarse label: {y_coarse_dict[a]}")
    print(f"    true coarse label:   {true_coarse[a]}\n")

argmax dictionary coarse accuracy: 0.5000

[0] predict fine label:    90
    true fine label:      49
    predict coarse label: 18
    true coarse label:   9

[1] predict fine label:    80
    true fine label:      33
    predict coarse label: 16
    true coarse label:   6

[2] predict fine label:    79
    true fine label:      72
    predict coarse label: 15
    true coarse label:   14

[3] predict fine label:    4
    true fine label:      51
    predict coarse label: 0
    true coarse label:   10

[4] predict fine label:    71
    true fine label:      71
    predict coarse label: 14
    true coarse label:   14

[5] predict fine label:    79
    true fine label:      92
    predict coarse label: 15
    true coarse label:   18

[6] predict fine label:    15
    true fine label:      15
    predict coarse label: 3
    true coarse label:   3

[7] predict fine label:    7
    true fine label:      14
    predict coarse label: 1
    true coarse label:   2

[8] predict fine label:    71


## 2. condfidence 기반 매핑

In [14]:
import numpy as np

def entropy_based_coarse(softmax_output, fine_to_coarse, num_coarse=20):
    """
    softmax 결과의 엔트로피를 기반으로 신뢰도를 추정하고,
    coarse-class 확률로 변환하는 후처리 함수입니다.

    Parameters:
    -----------
    softmax_output : np.ndarray
        shape (N, 100) — 모델의 fine-class softmax 출력
    fine_to_coarse : dict
        fine-class index (0~99) → coarse-class index (0~19) 매핑 딕셔너리
    num_coarse : int
        coarse 클래스 개수 (기본: 20)

    Returns:
    --------
    coarse_probs : np.ndarray
        shape (N, 20) — 각 sample의 coarse-class 확률 분포
    """

    N = softmax_output.shape[0]  # sample 개수
    log100 = np.log(100)         # 최대 엔트로피 값 (정규화용)
    coarse_probs = np.zeros((N, num_coarse))  # 결과 coarse 확률 초기화

    for i in range(N):
        # ① 해당 샘플의 softmax 벡터 (fine-class 확률)
        fine_probs = softmax_output[i]  # shape (100,)

        # ② entropy 계산 (불확실성 측정)
        entropy = -np.sum(fine_probs * np.log(fine_probs + 1e-9))  # log(0) 방지

        # ③ 신뢰도 계산: 확신이 높을수록 1에 가까움
        confidence = 1 - (entropy / log100)

        # ④ fine-class 확률에 confidence 곱하기 (확신도 기반 가중)
        weighted_fine_probs = fine_probs * confidence  # shape (100,)

        # ⑤ coarse-class 단위로 합산
        for fine_idx, prob in enumerate(weighted_fine_probs):
            coarse_idx = fine_to_coarse[fine_idx]
            coarse_probs[i][coarse_idx] += prob

    return coarse_probs  # shape (N, 20)



In [15]:
# 1. 모델의 softmax 출력 (fine-class 기준)
##softmax

# 2. fine → coarse 매핑 딕셔너리
fine_to_coarse = get_cifar100_fine_to_coarse_dict()

# 3. coarse-class 확률 얻기 (confidence 가중합 방식)
coarse_confidence_weight = entropy_based_coarse(softmax_output, fine_to_coarse)

# 4. coarse-class 예측 (argmax)
y_coarse_confi = np.argmax(coarse_confidence_weight, axis=1)


# --- 정확도 출력 ---
acc_confidence = np.mean(y_coarse_confi == true_coarse)
print(f"confidence coarse accuracy: {acc_confidence:.4f}\n")

# --- 예시 출력 ---
for a in range(30):
    print(f"[{a}] predict fine label:    {pred_fine[a]}")
    print(f"    true fine label:      {true_fine[a]}")
    print(f"    predict coarse label: {y_coarse_confi[a]}")
    print(f"    true coarse label:   {true_coarse[a]}\n")

confidence coarse accuracy: 0.5020

[0] predict fine label:    90
    true fine label:      49
    predict coarse label: 18
    true coarse label:   9

[1] predict fine label:    80
    true fine label:      33
    predict coarse label: 16
    true coarse label:   6

[2] predict fine label:    79
    true fine label:      72
    predict coarse label: 9
    true coarse label:   14

[3] predict fine label:    4
    true fine label:      51
    predict coarse label: 5
    true coarse label:   10

[4] predict fine label:    71
    true fine label:      71
    predict coarse label: 14
    true coarse label:   14

[5] predict fine label:    79
    true fine label:      92
    predict coarse label: 15
    true coarse label:   18

[6] predict fine label:    15
    true fine label:      15
    predict coarse label: 3
    true coarse label:   3

[7] predict fine label:    7
    true fine label:      14
    predict coarse label: 1
    true coarse label:   2

[8] predict fine label:    71
    true

## 3.soft label 기반 coarse 매핑

In [16]:
def softlabel_coarse_mapping(fine_probs, fine_to_coarse_dict, num_coarse=20):
    batch_size = fine_probs.shape[0]
    coarse_probs = np.zeros((batch_size, num_coarse))

    for fine_idx in range(100):
        coarse_idx = fine_to_coarse_dict[fine_idx]
        coarse_probs[:, coarse_idx] += fine_probs[:, fine_idx]
    
    return coarse_probs

In [17]:
# 1. 모델의 softmax 출력값 얻기
#softmax

# 2. fine → coarse 매핑 딕셔너리 준비 (0~99 → 0~19)
fine_to_coarse = get_cifar100_fine_to_coarse_dict()

# 3. soft label 기반 coarse 확률 계산
coarse_probs = softlabel_coarse_mapping(softmax_output, fine_to_coarse)
# → shape (N, 20), fine-class 확률들을 coarse-class별로 집계

# 4. coarse label 예측 (확률 중 가장 높은 값 선택)
y_coarse_soft = np.argmax(coarse_probs, axis=1)

# --- 정확도 출력 ---
acc_soft = np.mean(y_coarse_soft == true_coarse)
print(f"soft label coarse accuracy: {acc_soft:.4f}\n")

# --- 예시 출력 ---
for a in range(30):
    print(f"[{a}] predict fine label:    {pred_fine[a]}")
    print(f"    true fine label:      {true_fine[a]}")
    print(f"    predict coarse label: {y_coarse_soft[a]}")
    print(f"    true coarse label:   {true_coarse[a]}\n")

soft label coarse accuracy: 0.5020

[0] predict fine label:    90
    true fine label:      49
    predict coarse label: 18
    true coarse label:   9

[1] predict fine label:    80
    true fine label:      33
    predict coarse label: 16
    true coarse label:   6

[2] predict fine label:    79
    true fine label:      72
    predict coarse label: 9
    true coarse label:   14

[3] predict fine label:    4
    true fine label:      51
    predict coarse label: 5
    true coarse label:   10

[4] predict fine label:    71
    true fine label:      71
    predict coarse label: 14
    true coarse label:   14

[5] predict fine label:    79
    true fine label:      92
    predict coarse label: 15
    true coarse label:   18

[6] predict fine label:    15
    true fine label:      15
    predict coarse label: 3
    true coarse label:   3

[7] predict fine label:    7
    true fine label:      14
    predict coarse label: 1
    true coarse label:   2

[8] predict fine label:    71
    true

## 4. Validation-guided fine class weighting for coarse mapping

In [18]:
#Step 1. fine-class별 validation 정확도 계산 → fine_weight[100]
#Step 2. softmax_output × fine_weight → weighted_softmax
#Step 3. fine → coarse 그룹 합산 → coarse_probs_weighted
#Step 4. argmax(coarse_probs_weighted) → coarse 예측

def compute_fine_class_accuracy(pred_fine, true_fine, num_classes=100):
    correct = np.zeros(num_classes)
    total = np.zeros(num_classes)

    for pf, tf in zip(pred_fine, true_fine):
        total[tf] += 1
        if pf == tf:
            correct[tf] += 1

    acc = correct / (total + 1e-9)  # avoid divide by 0
    acc = acc / np.max(acc)        # normalize to [0, 1]
    return acc  # shape (100,)

def validation_weighted_coarse(softmax_output, fine_weight, fine_to_coarse, num_coarse=20):
    weighted_fine = softmax_output * fine_weight[np.newaxis, :]  # shape (N, 100)
    coarse_probs = np.zeros((softmax_output.shape[0], num_coarse))
    
    for f in range(100):
        c = fine_to_coarse[f]
        coarse_probs[:, c] += weighted_fine[:, f]
    
    return coarse_probs  # shape (N, 20)

In [19]:
# 1. 모델 예측 수행
#softmax_output

# 2. fine label 예측 (argmax)
pred_valid_fine= np.argmax(softmax_output, axis=1)
# 3. fine label 정답값
true_valid_fine = t_valid_fine  # shape (N,)
# 4. fine-class별 validation 정확도 계산 (normalize된 신뢰도 weight)
valid_weight = compute_fine_class_accuracy(pred_valid_fine, true_valid_fine)
# 5. coarse 확률 계산 (fine softmax × fine weight → coarse 집계)
coarse_probs = validation_weighted_coarse(softmax_output, valid_weight, fine_to_coarse)

# 6. coarse 예측
y_coarse_valid = np.argmax(coarse_probs, axis=1)

# --- 정확도 출력 ---
acc_valid = np.mean(y_coarse_valid == true_coarse)
print(f"Validation-weighted coarse accuracy: {acc_valid:.4f}\n")

# --- 예시 출력 ---
for a in range(30):
    print(f"[{a}] predict fine label:    {pred_fine[a]}")
    print(f"    true fine label:      {true_fine[a]}")
    print(f"    predict coarse label: {y_coarse_valid[a]}")
    print(f"    true coarse label:   {true_coarse[a]}\n")

Validation-weighted coarse accuracy: 0.2512

[0] predict fine label:    90
    true fine label:      49
    predict coarse label: 9
    true coarse label:   9

[1] predict fine label:    80
    true fine label:      33
    predict coarse label: 11
    true coarse label:   6

[2] predict fine label:    79
    true fine label:      72
    predict coarse label: 15
    true coarse label:   14

[3] predict fine label:    4
    true fine label:      51
    predict coarse label: 15
    true coarse label:   10

[4] predict fine label:    71
    true fine label:      71
    predict coarse label: 14
    true coarse label:   14

[5] predict fine label:    79
    true fine label:      92
    predict coarse label: 15
    true coarse label:   18

[6] predict fine label:    15
    true fine label:      15
    predict coarse label: 5
    true coarse label:   3

[7] predict fine label:    7
    true fine label:      14
    predict coarse label: 15
    true coarse label:   2

[8] predict fine label:    

## 전체 정확도

In [24]:
print(f"[🎯 Fine Label] Test Accuracy: {fine_test_acc:.4f}\n")

print(f"argmax dictionary coarse accuracy: {acc_dict:.4f}\n")
print(f"confidence coarse accuracy: {acc_confidence:.4f}\n")
print(f"soft label coarse accuracy: {acc_soft:.4f}\n")
print(f"Validation-weighted coarse accuracy: {acc_valid:.4f}\n")

[🎯 Fine Label] Test Accuracy: 0.4826

argmax dictionary coarse accuracy: 0.5000

confidence coarse accuracy: 0.5020

soft label coarse accuracy: 0.5020

Validation-weighted coarse accuracy: 0.2512

